# Walk Forward: A Realistic Approach to Backtesting

In [ ]:
???

![](<src/10_Table_Validation Methods.png>)

## Load the data

In [2]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', parse_dates=['Date'], index_col=0)
df = df.drop(columns='change_tomorrow_direction')
df

,Open,High,Low,Close,Adj Close,Volume,change_tomorrow
Date,,,,,,,
2016-12-08,61.299999,61.580002,60.840000,61.009998,55.705235,21220800,1.549141
2016-12-09,61.180000,61.990002,61.130001,61.970001,56.581772,27349400,0.321694
2016-12-12,61.820000,62.299999,61.720001,62.169998,56.764374,20198100,1.286125
2016-12-13,62.500000,63.419998,62.240002,62.980000,57.503944,35718900,-0.478620
2016-12-14,63.000000,63.450001,62.529999,62.680000,57.230022,30352700,-0.159793
...,...,...,...,...,...,...,...
2023-12-18,369.450012,373.000000,368.679993,372.649994,372.649994,21802900,0.163429
2023-12-19,371.489990,373.260010,369.839996,373.260010,373.260010,20603700,-0.712324
2023-12-20,375.000000,376.029999,370.529999,370.619995,370.619995,26316700,0.781714


## Walk Forward Validation

### How `TimeSeriesSplit` works

In [1]:
from sklearn.model_selection import TimeSeriesSplit

In [3]:
ts = TimeSeriesSplit(test_size=200)
splits = ts.split(X=df)

In [5]:
split2 = next(splits)
split2

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [6]:
list_df_train = []
list_df_test = []

for index_train, index_test in ts.split(X=df):
    list_df_train.append(df.iloc[index_train])
    list_df_test.append(df.iloc[index_test])

## Machine Learning Model

### Separate the data

1. Target: which variable do you want to predict?
2. Explanatory: which variables will you use to calculate the prediction?

In [7]:
y = df.change_tomorrow
X = df[['Open','High','Low','Close','Volume']]

In [8]:
list_df_train = []
list_df_test = []

for index_train, index_test in ts.split(df):
    X_train, y_train = X.iloc[index_train], y.iloc[index_train]
    X_test, y_test = X.iloc[index_test], y.iloc[index_test]

### Simulate one computation of the ML model

- Compute the model
- Calculate predictions on the test set
- Evaluate how good the model is

In [9]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

model_dt = DecisionTreeRegressor(max_depth=15, random_state=42)
model_dt.fit(X_train, y_train)

y_pred = model_dt.predict(X_test)
error_mse = mean_squared_error(y_test, y_pred)
error_mse

3.4219764962040595

### Add the procedure inside the for loop

In [10]:
model_dt = DecisionTreeRegressor(max_depth=15, random_state=42)

error_mse_list = []

for index_train, index_test in ts.split(df):
    
    X_train, y_train = X.iloc[index_train], y.iloc[index_train]
    X_test, y_test = X.iloc[index_test], y.iloc[index_test]

    model_dt.fit(X_train, y_train)

    y_pred = model_dt.predict(X_test)
    error_mse = mean_squared_error(y_test, y_pred)

    error_mse_list.append(error_mse)

In [11]:
error_mse_list

[12.163689658106115,
 33.69213503324955,
 7.237922690463958,
 5.342406443574685,
 3.4219764962040595]

In [12]:
import numpy as np
np.mean(error_mse_list)

12.371626064319674

## Anchored Walk Forward evaluation in backtesting

![](<src/10_Table_Validation Methods.png>)

### Create a new strategy

In [18]:
from backtesting import Backtest, Strategy

In [19]:
bt = Backtest(df, Regression, cash=10000, commission=.002, exclusive_orders=True)

In [20]:
class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    n_train = 600
    coeff_retrain = 200
    
    def init(self):
        self.model = DecisionTreeRegressor(max_depth=15, random_state=42)
        self.already_bought = False
        
        X_train = self.data.df.iloc[:self.n_train, :-1]
        y_train = self.data.df.iloc[:self.n_train, -1]
        
        self.model.fit(X=X_train, y=y_train)

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :-1]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [21]:
class WalkForwardAnchored(Regression):
    def next(self):
        
        # we don't take any action and move on to the following day
        if len(self.data) < self.n_train:
            return
        
        # we retrain the model each x days
        if len(self.data) % self.coeff_retrain == 0:
            X_train = self.data.df.iloc[:, :-1]
            y_train = self.data.df.iloc[:, -1]

            self.model.fit(X_train, y_train)

            super().next()
            
        else:
            
            super().next()

In [22]:
from backtesting import Backtest
bt = Backtest(df, WalkForwardAnchored, cash=10000, commission=.002, exclusive_orders=True)

In [23]:
import multiprocessing as mp
mp.set_start_method('fork')

In [ ]:
stats_skopt, heatmap, optimize_result = bt.optimize(
    limit_buy = range(0, 6), limit_sell = range(-6, 0),
    maximize='Return [%]',
    max_tries=500,
    random_state=42,
    return_heatmap=True,
    return_optimization=True,
    method='skopt'
    )

dff = heatmap.reset_index()
dff = dff.sort_values('Return [%]', ascending=False)
dff

## Unanchored Walk Forward

### Create a library of strategies

`strategies.py`

### Create the unanchored walk forward class

![](<src/10_Table_Validation Methods.png>)

### Import the strategy and perform the backtest

In [25]:
%load_ext autoreload
%autoreload 2

In [26]:
import strategies

In [27]:
strategies.WalkForwardUnanchored

strategies.WalkForwardUnanchored

In [ ]:
bt_unanchored = Backtest(df, strategies.WalkForwardUnanchored, cash=10000, commission=.002, exclusive_orders=True)

stats_skopt, heatmap, optimize_result = bt_unanchored.optimize(
    limit_buy = range(0, 6), limit_sell = range(-6, 0),
    maximize='Return [%]',
    max_tries=500,
    random_state=42,
    return_heatmap=True,
    return_optimization=True,
    method='skopt'
    )

dff = heatmap.reset_index()
dff = dff.sort_values('Return [%]', ascending=False)
dff

### Interpret the strategies' performance

- Both anchored and unanchored backtesting

In [ ]:
bt.plot(filename='reports_backtesting/walk_forward_anchored.html')

In [ ]:
bt_unanchored.plot(filename='reports_backtesting/walk_forward_unanchored.html')

## Practice to master the knowledge

Work on the challenge with another dataset:

1. Learn the <a>mental models</a> to solve the challenge faster.
2. Complete the <a href="10C_Walk Forward Regression.ipynb">notebook</a>.